In [1]:
import numpy as np

import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
import keras.backend as K

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('data/sentipolc_word_index.json', 'r') as f:
    word_index = json.load(f)

In [3]:
with open('data/words.tsv', 'w') as f:
    for w,_ in word_index.items():
        f.write('{}\n'.format(w))

In [4]:
words = [w for w,_ in word_index.items()]

In [5]:
data = np.load('data/sentipolc_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

Check the max length of the text

In [6]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(word_index.values())
print('max id {}'.format(max_idx))

max len seq 40
max id 10218


In [7]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq)
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq)

In [8]:
x_train_pad.shape

(4154, 40)

# Define metrics

In [9]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
     how many selected items are relevant.
      """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [10]:
def recall(y_true, y_pred):
    """Recall metric.
 
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [11]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_test = StratifiedShuffleSplit(1,test_size=0.2, random_state=45)

for train, test in split_train_test.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [12]:
callbacks = [
    ReduceLROnPlateau(patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
]

In [17]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx+1, output_dim=50, input_length=max_len_seq))
    model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu', strides=1))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu', strides=1))
    model.add(GlobalMaxPool1D())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [18]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 50)            510950    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 40, 64)            16064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 20, 64)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 20, 32)            10272     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                1650      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
__________

In [19]:
history = model.fit(x_train_pad_split, y_train_pad_split, validation_split=0.2, batch_size=32, epochs=10, callbacks=callbacks)

Train on 2658 samples, validate on 665 samples
Epoch 1/10
2658/2658 [==============================] - 2s 834us/step - loss: 0.6599 - acc: 0.6166 - val_loss: 0.6455 - val_acc: 0.6526
Epoch 2/10
2658/2658 [==============================] - 0s 155us/step - loss: 0.5480 - acc: 0.7302 - val_loss: 0.5416 - val_acc: 0.7368
Epoch 3/10
2658/2658 [==============================] - 0s 156us/step - loss: 0.2641 - acc: 0.9063 - val_loss: 0.5821 - val_acc: 0.7308
Epoch 4/10
2658/2658 [==============================] - 0s 155us/step - loss: 0.0914 - acc: 0.9737 - val_loss: 0.8604 - val_acc: 0.7218
Epoch 5/10
2658/2658 [==============================] - 0s 156us/step - loss: 0.0330 - acc: 0.9944 - val_loss: 1.1010 - val_acc: 0.7143

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 6/10
2658/2658 [==============================] - 0s 155us/step - loss: 0.0142 - acc: 0.9970 - val_loss: 1.1222 - val_acc: 0.7233
Epoch 7/10
2658/2658 [==============================] -

## Evaluate on the test set

In [20]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=2)

Epoch 1/2
4154/4154 [==============================] - 1s 145us/step - loss: 0.6743 - acc: 0.6021
Epoch 2/2
4154/4154 [==============================] - 0s 43us/step - loss: 0.6293 - acc: 0.6249


In [21]:
model.evaluate(x_test_pad, y_test)

1050/1050 [==============================] - 0s 107us/step


[0.5683503635724385, 0.7342857141721816]

In [ ]:
#model.save('models/sentipolc_word_cnn.hdf5')

# Overall metrics

In [22]:
from sklearn import metrics

In [23]:
predictions = model.predict_classes(x_train_pad, batch_size=128)
tn, fp, fn, tp = metrics.confusion_matrix(y_train, predictions).ravel()

recall = tp/(tp+fp)
precision = tp/(tp+fn)
f1 = metrics.f1_score(y_train,predictions)

print ('precision: {}'.format(precision), '\nrecall: {}'.format(recall), '\nf1: {}'.format(f1))

precision: 0.2756052141527002 
recall: 0.8792079207920792 
f1: 0.4196597353497164
